## 패키지 불러오기

In [ ]:
%%time 

!apt install gdal-bin python-gdal python3-gdal 
!apt install python3-rtree 
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes 
!pip install folium
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import plotly_express as px

## 인구통계 데이터 열어보기

In [ ]:
population = pd.read_csv('07.인구통계.csv')

In [ ]:
population.tail(3)

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
1581,경기도 고양시 일산서구 송산동(4128759000),2019년10월,"50,660","18,398",2.75,"25,020","25,640",0.98
1582,경기도 고양시 일산서구 송산동(4128759000),2019년11월,"50,865","18,480",2.75,"25,112","25,753",0.98
1583,경기도 고양시 일산서구 송산동(4128759000),2019년12월,"51,060","18,555",2.75,"25,195","25,865",0.97


In [ ]:
# 가장 최근 것만 가져온다

population_latest = population.loc[population['조회기준'].str.contains('2019년12월', na=False)]
population_latest

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
35,경기도 고양시 (4128000000),2019년12월,"1,066,351","434,028",2.46,"522,749","543,602",0.96
71,경기도 고양시 덕양구 (4128100000),2019년12월,"466,157","195,766",2.38,"230,858","235,299",0.98
107,경기도 고양시 덕양구 주교동(4128151000),2019년12월,"15,748","7,362",2.14,"8,126","7,622",1.07
143,경기도 고양시 덕양구 원신동(4128152000),2019년12월,"23,942","9,419",2.54,"11,568","12,374",0.93
179,경기도 고양시 덕양구 흥도동(4128153000),2019년12월,"44,565","18,311",2.43,"21,702","22,863",0.95
215,경기도 고양시 덕양구 성사1동(4128154000),2019년12월,"22,062","9,676",2.28,"10,928","11,134",0.98
251,경기도 고양시 덕양구 성사2동(4128155000),2019년12월,"11,729","4,783",2.45,"5,689","6,040",0.94
287,경기도 고양시 덕양구 효자동(4128156000),2019년12월,"5,041","2,480",2.03,"2,546","2,495",1.02
323,경기도 고양시 덕양구 신도동(4128157000),2019년12월,0,0,0.00,0,0,0.00
359,경기도 고양시 덕양구 삼송동(4128157500),2019년12월,"21,879","9,660",2.26,"10,647","11,232",0.95


In [ ]:
# 고양시 전체인구와 구별 인구를 날려준다

population_latest2 = population_latest.copy()
population_latest2.drop([35,71,827,1259], inplace=True)

In [ ]:
population_latest2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 107 to 1583
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   행정구역    40 non-null     object 
 1   조회기준    40 non-null     object 
 2   총인구수    40 non-null     object 
 3   세대수     40 non-null     object 
 4   세대당 인구  40 non-null     float64
 5   남자 인구수  40 non-null     object 
 6   여자 인구수  40 non-null     object 
 7   남여 비율   40 non-null     float64
dtypes: float64(2), object(6)
memory usage: 2.8+ KB


In [ ]:
# 신도동 = 삼송동 (2018년에 개칭했다고 함) => 날려줘야함
# https://namu.wiki/w/%EC%82%BC%EC%86%A1%EB%8F%99

population_latest3 = population_latest2.copy()
population_latest3.drop(323, inplace=True)

In [ ]:
population_latest3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 107 to 1583
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   행정구역    39 non-null     object 
 1   조회기준    39 non-null     object 
 2   총인구수    39 non-null     object 
 3   세대수     39 non-null     object 
 4   세대당 인구  39 non-null     float64
 5   남자 인구수  39 non-null     object 
 6   여자 인구수  39 non-null     object 
 7   남여 비율   39 non-null     float64
dtypes: float64(2), object(6)
memory usage: 2.7+ KB


## 행정동 경계 파일 열어보기

In [ ]:
# 행정동 경계

hang = gpd.read_file('34.고양시_행정경계(행정동기준).geojson')
hang.head(3)

,행정동코드,행정동명,geometry
0,3110151,주교동,"MULTIPOLYGON (((126.83514 37.67683, 126.83517 ..."
1,3110152,원신동,"MULTIPOLYGON (((126.84983 37.68986, 126.84996 ..."
2,3110153,흥도동,"MULTIPOLYGON (((126.87615 37.64768, 126.87562 ..."


In [ ]:
hang.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   행정동코드     39 non-null     object  
 1   행정동명      39 non-null     object  
 2   geometry  39 non-null     geometry
dtypes: geometry(1), object(2)
memory usage: 1.0+ KB


## 인구통계 자료에 경위도값 더해주기

In [ ]:
# 인구통계의 '행정구역'과 행정동의 '행정동명'이 일치해야한다

population_latest4 = population_latest3.copy()
population_latest4['행정구역'] = population_latest4['행정구역'].str[12:].str[-16:-12].str.replace(' ','')

In [ ]:
population_latest4

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
107,주교동,2019년12월,"15,748","7,362",2.14,"8,126","7,622",1.07
143,원신동,2019년12월,"23,942","9,419",2.54,"11,568","12,374",0.93
179,흥도동,2019년12월,"44,565","18,311",2.43,"21,702","22,863",0.95
215,성사1동,2019년12월,"22,062","9,676",2.28,"10,928","11,134",0.98
251,성사2동,2019년12월,"11,729","4,783",2.45,"5,689","6,040",0.94
287,효자동,2019년12월,"5,041","2,480",2.03,"2,546","2,495",1.02
359,삼송동,2019년12월,"21,879","9,660",2.26,"10,647","11,232",0.95
395,창릉동,2019년12월,"19,852","8,515",2.33,"9,970","9,882",1.01
431,고양동,2019년12월,"31,168","12,763",2.44,"15,651","15,517",1.01
467,관산동,2019년12월,"36,611","16,182",2.26,"18,644","17,967",1.04


In [ ]:
# 이제 경위도값 입력

import warnings
warnings.filterwarnings(action='ignore') # 경고메시지가 떠서 설치해줌

population_latest4['geometry'] = 0

for i in range(len(population_latest4['행정구역'])):
    for j in range(len(hang['행정동명'])):
        if population_latest4['행정구역'].iloc[i] == hang['행정동명'].iloc[j] :
            population_latest4['geometry'].iloc[i] = hang['geometry'].iloc[j]
            break

In [ ]:
population_latest4

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,geometry
107,주교동,2019년12월,"15,748","7,362",2.14,"8,126","7,622",1.07,(POLYGON ((126.8351382691004 37.67683481916347...
143,원신동,2019년12월,"23,942","9,419",2.54,"11,568","12,374",0.93,(POLYGON ((126.8498343812966 37.68985923284904...
179,흥도동,2019년12월,"44,565","18,311",2.43,"21,702","22,863",0.95,(POLYGON ((126.8761498094425 37.64767924861452...
215,성사1동,2019년12월,"22,062","9,676",2.28,"10,928","11,134",0.98,(POLYGON ((126.8489798158804 37.66872091518727...
251,성사2동,2019년12월,"11,729","4,783",2.45,"5,689","6,040",0.94,"(POLYGON ((126.8350241399075 37.6549071786553,..."
287,효자동,2019년12월,"5,041","2,480",2.03,"2,546","2,495",1.02,(POLYGON ((126.9573625714565 37.69194768044995...
359,삼송동,2019년12월,"21,879","9,660",2.26,"10,647","11,232",0.95,(POLYGON ((126.9063025510095 37.67545134414731...
395,창릉동,2019년12월,"19,852","8,515",2.33,"9,970","9,882",1.01,(POLYGON ((126.9007203787597 37.65079259143342...
431,고양동,2019년12월,"31,168","12,763",2.44,"15,651","15,517",1.01,(POLYGON ((126.9290958517618 37.74848500799976...
467,관산동,2019년12월,"36,611","16,182",2.26,"18,644","17,967",1.04,(POLYGON ((126.8589108100696 37.73417369660056...


In [ ]:
# QGIS에서 value값에 따라 시각화를 해보려면 데이터타입이 숫자형이어야 한다

population_latest4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 107 to 1583
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   행정구역      39 non-null     object 
 1   조회기준      39 non-null     object 
 2   총인구수      39 non-null     object 
 3   세대수       39 non-null     object 
 4   세대당 인구    39 non-null     float64
 5   남자 인구수    39 non-null     object 
 6   여자 인구수    39 non-null     object 
 7   남여 비율     39 non-null     float64
 8   geometry  39 non-null     object 
dtypes: float64(2), object(7)
memory usage: 3.0+ KB


In [ ]:
# 총인구수, 세대수, 남자 인구수, 여자 인구수를 숫자형으로 변환

population_latest5 = population_latest4.copy()

population_latest5["총인구수"]= population_latest5["총인구수"].str.replace(',','')
population_latest5["세대수"]= population_latest5["세대수"].str.replace(',','')
population_latest5["남자 인구수"]= population_latest5["남자 인구수"].str.replace(',','')
population_latest5["여자 인구수"]= population_latest5["여자 인구수"].str.replace(',','')


population_latest5["총인구수"]= population_latest5["총인구수"].astype(int)
population_latest5["세대수"]= population_latest5["세대수"].astype(int)
population_latest5["남자 인구수"]= population_latest5["남자 인구수"].astype(int)
population_latest5["여자 인구수"]= population_latest5["여자 인구수"].astype(int)

In [ ]:
population_latest5.head(3)

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,geometry
107,주교동,2019년12월,15748,7362,2.14,8126,7622,1.07,(POLYGON ((126.8351382691004 37.67683481916347...
143,원신동,2019년12월,23942,9419,2.54,11568,12374,0.93,(POLYGON ((126.8498343812966 37.68985923284904...
179,흥도동,2019년12월,44565,18311,2.43,21702,22863,0.95,(POLYGON ((126.8761498094425 37.64767924861452...


In [ ]:
# 이 파일을 csv로 저장

population_latest5.to_csv('population_latest5.csv')

## 구별로 묶어봐도 좋을듯

In [ ]:
population_latest2 = population.loc[population['조회기준'].str.contains('2019년12월', na=False)]
population_latest2

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
35,경기도 고양시 (4128000000),2019년12월,"1,066,351","434,028",2.46,"522,749","543,602",0.96
71,경기도 고양시 덕양구 (4128100000),2019년12월,"466,157","195,766",2.38,"230,858","235,299",0.98
107,경기도 고양시 덕양구 주교동(4128151000),2019년12월,"15,748","7,362",2.14,"8,126","7,622",1.07
143,경기도 고양시 덕양구 원신동(4128152000),2019년12월,"23,942","9,419",2.54,"11,568","12,374",0.93
179,경기도 고양시 덕양구 흥도동(4128153000),2019년12월,"44,565","18,311",2.43,"21,702","22,863",0.95
215,경기도 고양시 덕양구 성사1동(4128154000),2019년12월,"22,062","9,676",2.28,"10,928","11,134",0.98
251,경기도 고양시 덕양구 성사2동(4128155000),2019년12월,"11,729","4,783",2.45,"5,689","6,040",0.94
287,경기도 고양시 덕양구 효자동(4128156000),2019년12월,"5,041","2,480",2.03,"2,546","2,495",1.02
323,경기도 고양시 덕양구 신도동(4128157000),2019년12월,0,0,0.00,0,0,0.00
359,경기도 고양시 덕양구 삼송동(4128157500),2019년12월,"21,879","9,660",2.26,"10,647","11,232",0.95


In [ ]:
population_latest2_gu = population_latest2.loc[[71,827,1259], :]
population_latest2_gu

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
71,경기도 고양시 덕양구 (4128100000),2019년12월,"466,157","195,766",2.38,"230,858","235,299",0.98
827,경기도 고양시 일산동구 (4128500000),2019년12월,"297,671","122,981",2.42,"144,991","152,680",0.95
1259,경기도 고양시 일산서구 (4128700000),2019년12월,"302,523","115,281",2.62,"146,900","155,623",0.94


In [ ]:
# 행정구역 이름을 맞춰준다

population_latest2_gu2 = population_latest2_gu.copy()

population_latest2_gu2['행정구역'] = population_latest2_gu2['행정구역'].str.replace(' ','').str[3:].str[-19:-12]
population_latest2_gu2

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
71,고양시덕양구,2019년12월,"466,157","195,766",2.38,"230,858","235,299",0.98
827,고양시일산동구,2019년12월,"297,671","122,981",2.42,"144,991","152,680",0.95
1259,고양시일산서구,2019년12월,"302,523","115,281",2.62,"146,900","155,623",0.94


In [ ]:
# 시군구 데이터 열기

sgg = gpd.read_file('08.행정경계(시군구).geojson')
sgg.head(3)

,SIG_CD,SIG_KOR_NM,geometry
0,41281,고양시덕양구,"MULTIPOLYGON (((126.92910 37.74849, 126.92964 ..."
1,41285,고양시일산동구,"MULTIPOLYGON (((126.77762 37.67899, 126.77825 ..."
2,41287,고양시일산서구,"MULTIPOLYGON (((126.76703 37.70717, 126.76719 ..."


In [ ]:
# 엮자

population_latest2_gu2['geometry'] = 0

for i in range(len(population_latest2_gu['행정구역'])):
    for j in range(len(sgg['SIG_KOR_NM'])):
        if population_latest2_gu2['행정구역'].iloc[i] == sgg['SIG_KOR_NM'].iloc[j] :
            population_latest2_gu2['geometry'].iloc[i] = sgg['geometry'].iloc[j]
            break

In [ ]:
# 확인

population_latest2_gu2

,행정구역,조회기준,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,geometry
71,고양시덕양구,2019년12월,"466,157","195,766",2.38,"230,858","235,299",0.98,(POLYGON ((126.9290964179607 37.74848500855844...
827,고양시일산동구,2019년12월,"297,671","122,981",2.42,"144,991","152,680",0.95,"(POLYGON ((126.7776157073954 37.6789946909556,..."
1259,고양시일산서구,2019년12월,"302,523","115,281",2.62,"146,900","155,623",0.94,(POLYGON ((126.7670325668513 37.70716707003747...


In [ ]:
# csv로 저장

population_latest2_gu2.to_csv('population_latest_gu2.csv')